# Custom unit operations

New and emerging processing technologies often require custom tailored unit operations. In this problem set, we will leverage BioSTEAM's extendable framework to model fermentation for ethanol production. Mainly, we will:

1. Inherit design and costing algorithms from the abstract StirredTankReactor class and specify the mass and energy balance.
2. Test our results.

**Step 1: Inherit from the StirredTankReactor class**

* Two outlets: vent, broth

* Operating temperature and pressure: 32 degC and 1 atm.

* Reaction time: 60 hr.

* Ethanol yield: 90 % theoretical.

* Cell growth: 70 % of remaining glucose by weight.


In [ ]:
import biosteam as bst
class Fermentation(bst.StirredTankReactor):
    _N_ins = 1 # Number of inlets
    _N_outs = None # Number of outlets
    T_default = None # Operating temperature [K]
    P_default = 101325. # Pressure[Pa]
    tau_default = None # Reaction time [hr]
    auxiliary_units = (
        'preheater', 
        *bst.StirredTankReactor.auxiliary_units
    )

    def _run(self):
        fermentation = None # Define reaction with bst.Reaction
        cell_growth = bst.Reaction('Glucose -> Yeast', 'Glucose', 0.70, basis='wt')
        
        feed, = self.ins # Get inlets 
        vent, effluent = None # Get outlets
        
        self.auxiliary(
            'preheater', bst.HXutility, 
            ins=feed, # Feed from parent 
            T=None # Outlet temperature
        )
        heated_feed = self.preheater.outs[0]
        effluent.copy_like(heated_feed)
        
        # Isothermally react effluent
        fermentation(effluent)
        cell_growth(effluent)
        
        # Perform vapor-liquid equilibrium to find vent
        effluent.vle(T=None, P=None)
        vent.copy_like(effluent['g'])
        effluent['g'].empty()


**Step 2: Test results**

* Check diagram
* Check streams
* Check design and cost

In [ ]:
from biorefineries.corn import create_chemicals
chemicals = create_chemicals()
bst.settings.set_thermo(chemicals)
slurry = bst.Stream(
    'slurry', 
    Water=71.24, 
    Glucose=20.66, 
    TriOlein=1.153, 
    Fiber=3.646, 
    SolubleProtein=1.248,
    InsolubleProtein=1.523, 
    Ash=0.4254, 
    Yeast=0.01096, 
    CaO=0.003109, 
    H2SO4=0.0313,
    NH3=0.05181,
    units='kg/hr'
)
R1 = Fermentation(
    ins=None, 
    outs=('CO2', 'product')
)
R1.simulate()
R1.diagram()
R1.show()
R1.results()